# Stanford Cars using Fast.AI

In [ ]:
!deactivate # if you're using any 
import os
import multiprocessing
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # Change if GPU is more than 1

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.vision import *
from fastai.metrics import error_rate
from fastai import *
from fastai.callbacks import *

import cv2 as cv
import numpy as np
import pandas as pd
import scipy.io as sio

torch.backends.cudnn.benchmark = True

## Preprocessing Image

In [ ]:
from PIL import Image, ImageOps
from matplotlib.pyplot import imshow
import numpy as np
import scipy.io as sio
import os
import glob
import cv2
import matplotlib.pyplot as plt
import csv
from IPython.display import clear_output
%matplotlib inline

In [ ]:
dir = './data-kaggle/car_data/'
train = 'train/'

In [ ]:
files = []
folders = [f for f in glob.glob(dir + train + "**/", recursive=True)]
folders.pop(0)

In [ ]:
for cat in range(len(folders)):
    curCat = folders[cat]
    for r, d, f in os.walk(curCat):
        for file in f:
            fileName = curCat + file
            strToFind = '.jpg'
            None if file.find(strToFind) == -1 else files.append(curCat + file)

In [1]:
csv_dir = './data-kaggle/'

In [ ]:
class_names = []
file_data = []

In [ ]:
with open(csv_dir + 'names.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        class_names.append(row[0])

In [ ]:
with open(csv_dir + 'anno_train.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=",")
    for row in csv_reader:
        file_data.append([row[0], int(row[2]), int(row[4]), int(row[1]), int(row[3])])

In [ ]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image

    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)

    else:
        r = width / float(w)
        dim = (width, int(h * r))
        
    return cv2.resize(image, dim, interpolation = inter)

In [ ]:
counter = 0
width_less = 0
triple_width = 0
double_width = 0
vertical = 0

for i in files:
    base_img = cv2.cvtColor(cv2.imread(i), cv2.COLOR_BGR2RGB)
    csv_dir = './data-kaggle/'
    class_names = []
    fs = []
    for n in file_data:
        if n[0] == i.split('/')[-1]:
            fs = n
    cropped_img = base_img[fs[1]:fs[2], fs[3]:fs[4]]
    cropped_img_resize = image_resize(cropped_img, height = 400)
    height, width, _ = cropped_img_resize.shape
    name_0 = dir_name_crop.split('.jpg')[0] + '_cropped_0' + '.jpg'
    cv2.imwrite(name_0, cropped_img_resize)
    
    if width > 400 and width > height:
        mid_width = int(width / 2 - 200 if width / 2 - 200 >= 0 else width / width)
        max_width = int(width - 400 if width >= 400 else width / width)

        cropped_1 = cropped_img_resize[0:height, 0:400]
        cropped_2 = cropped_img_resize[0:height, mid_width:mid_width + 400]
        cropped_3 = cropped_img_resize[0:height, max_width:width]
    
        dir_name = i.split('/')
        dir_name[3] = 'train'
        dir_name_crop = ''
        
        for i in dir_name:
            dir_name_crop += (str(i) + '/')
            
        dir_name_crop = dir_name_crop[:-1]

        name_1 = dir_name_crop.split('.jpg')[0] + '_cropped_1' + '.jpg'
        name_2 = dir_name_crop.split('.jpg')[0] + '_cropped_2' + '.jpg'
        name_3 = dir_name_crop.split('.jpg')[0] + '_cropped_3' + '.jpg'

        cv2.imwrite(name_1, cropped_1)
        cv2.imwrite(name_2, cropped_2)
        cv2.imwrite(name_3, cropped_3)
    
    print(str(counter + 1) + " / " + str(len(files)) + " have been cropped.")
    counter += 1
    clear_output(wait=True)

## Building Model

In [ ]:
tfms = get_transforms(
    do_flip=False, 
    flip_vert=True, 
    max_rotate=15.0
)

num_workers = multiprocessing.cpu_count()

**data_build** is the function that returns new image bunch,

**fit** is the function that run the fitting process,

**change_size** is the function that change image bunch sizes and replace learner's data with new ones.

In [ ]:
def data_build(size):
    data = ImageDataBunch.from_folder(
        'data-kaggle/car_data/train',
        train='train',
        valid_pct=.2,
        ds_tfms=tfms,
        size=(size, size),
        num_workers=num_workers,
        bs=32).normalize(imagenet_stats)
    
    return data

Inside **fit** function, we are doing a couple of things:

1. Setting callbacks prior to fitting
2. Finding the best learning rate and use three of it as an array for multiple learning rates.
3. Fitting (both one cycle or regular fitting)
4. Test-time Augmentation

In [ ]:
def fit(model, epoch=1, one_cycle=False):
    
    # Callbacks
    reduceLR = ReduceLROnPlateauCallback(model, mode='max', patience=3, factor=.9)
    showGraph = ShowGraph(model)
    
    # Learning Rate
    model.lr_find()
    model.recorder.plot(suggestion=True)
    lr = model.recorder.min_grad_lr
    min_grad_lr = (lr/100, lr/10, lr)
    
    # Fit
    if one_cycle == True:
        model.fit_one_cycle(epoch, min_grad_lr, callbacks=[reduceLR, showGraph])
    
    else:
        model.fit(epoch, min_grad_lr, callbacks=[reduceLR, showGraph])
        
    # Test-time Augmentation
    learn.purge()
    accuracy(*model.TTA())

In [ ]:
def change_size(model, size=224):
    data = data_build(size)
    learn.data = data

In [ ]:
model = models.resnet50

In [ ]:
learn = cnn_learner(
    data_224, 
    model, 
    ps=0.1,
    bn_final=True,
    pretrained=True,
    opt_func=AdamW,
    metrics=[accuracy, error_rate]).mixup()

## 224 x 224px on 3 epochs

In [ ]:
data_224 = data_build(224)

In [ ]:
fit(learn, 3, False)

## 280 * 280px on 6 cyclical epochs (prone to removal)

In [ ]:
change_size(learn, 280)

In [ ]:
learn.unfreeze()

In [ ]:
fit(learn, 4, False)

## 336 * 336px on 4 cyclical epochs

In [ ]:
change_size(learn, 336)

In [ ]:
learn.unfreeze()

In [ ]:
fit(learn, 4, False)

## 400 * 400 on 4 cyclical epochs

In [ ]:
change_size(learn, 400)

In [ ]:
learn.unfreeze()

In [ ]:
fit(learn, 4, False)

## Testing on test data

For testing on test data, make sure each labels are separated by folders, otherwise the current code will not work. Here are the steps taken to validate test data:

1. Change image size to 400 * 400
2. Set batch size to 32
3. Normalize

In [ ]:
test_tfms = get_transforms()

In [ ]:
data_test = ImageDataBunch.from_folder(
    'data-kaggle/car_data/test',
    valid_pct=0,
    ds_tfms=test_tfms,
    size=(460, 460),
    num_workers=8,
    bs=32).normalize(imagenet_stats)

The results printed after the validation has finished are (in order):

1. Loss
2. Accuracy
3. Error rate

In [ ]:
learn.validate(data_test.train_dl)

Thank you! 🙏